In [ ]:
import numpy as np
import pandas as pd
import os
import re
import random
from collections import defaultdict
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file = "./drive/MyDrive/NLP/general-tweets.txt"
with open(file) as f:
  twitter_corpus = f.readlines()
  # len(twitter_corpus)
twitter_corpus[:5]

['Bumping dj sefs mixtape nowww this is my music new skooooool\n',
 '#ieroween THE STORY OF IEROWEEN! THE VIDEO ->>>>>>>>>>>>>>>>>>>>>> http://bit.ly/2VFPAV <<<< JUST FOR FRANK !!! Ã§\n',
 'trick or treating at the mall today; ZOO! last year we had left-overs, this year we ran out!\n',
 "@Ussk81 PMSL!!! I try not to stare but I can't help it, like compulsive viewing!!\n",
 '@Sc0rpi0n676 btw - is there a remote chance i will see you later?\n']

In [ ]:
def Tokenizer(st = None):
    word_regex=r"#\S+|@\S+|Dr\.|Mr\.|Mrs\.|[0-9]+\.[0-9]+|\w*'t|\w*'s|\w+|'|,|\.|\w*'t\b|\(|\)|\\|\/|-|!\s+"
    url_regex = r"https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}"
    word_regex = url_regex + r"|" + word_regex
    
    hash_regex = r"#\S+"
    mention_regex = r"@\S+"
    
    inp = """
    #ieroween THE STORY OF IEROWEEN! THE
    VIDEO -»»»»»»»»»»» http://bit.ly/2VFPAV ««
    JUST FOR FRANK !!! ÃƒÂ§
    """
    if st != None:
        inp = st
    ret = re.findall(word_regex, inp)
    ret = [re.sub(url_regex, "<URL>", x) for x in ret]
    ret = [re.sub(hash_regex, "<HASHTAG>", x) for x in ret]
    ret = [re.sub(mention_regex, "<MENTION>", x) for x in ret]
    return [s.strip() for s in ret]

# def Tokenizer(st = None):
#   url_regex = r"https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}"

#   sent=re.sub(url_regex,'<URL>',st)
#   sent=re.sub("#(\w+)","<HASTAG>",sent)
#   sent=re.sub("@(\w+)","<MENTION>",sent)
#   string="#@$%><*&/?.!~`-_=|.:;,"
#   for i in string:
#     reg='\\'
#     reg+=i
#     reg+=' + '
#     sent=re.sub(reg,i,sent)
#   for i in string:
#     reg='\\'
#     reg+=i
#     reg+='+'
#     sent=re.sub(reg,i,sent)
#   # print(sent)
#   # line=re.findall(word_regex,sent)
#   return sent.split()

In [ ]:
# Checking on twitter corpus
check = Tokenizer(twitter_corpus[1])
print(twitter_corpus[1])
print(check)

#ieroween THE STORY OF IEROWEEN! THE VIDEO ->>>>>>>>>>>>>>>>>>>>>> http://bit.ly/2VFPAV <<<< JUST FOR FRANK !!! Ã§

['<HASHTAG>', 'THE', 'STORY', 'OF', 'IEROWEEN', '!', 'THE', 'VIDEO', '-', '<URL>', 'JUST', 'FOR', 'FRANK', '!', 'Ã']


In [98]:
# Storing tokenized output of general-tweets into 2019111031_tokenize.txt
file = "./drive/MyDrive/NLP/general-tweets.txt"
with open(file) as f:
  data = f.readlines()

out = ""
for i, line in enumerate(data):
    data[i] = " ".join(Tokenizer(line))
    # print(data[i])
    out += (data[i]+"\n")
    
file1 = open("2019111031_tokenize.txt","w")
file1.write(out)
file1.close()

In [ ]:
# class stores info about the corpus
class Corpus:
  def __init__(self, corpus):
    self.total_words = 0
    self.test_sentences = []
    self.train_sentences = []
    self.test_indexes = random.sample(range(len(corpus)), 1000)
    self.corpus = corpus
    self.unique_history_count = defaultdict(lambda: 0)  # same as a {}, except defaultdict initializes with 0
    self.gramCount = defaultdict(lambda: 0)             # map mp[I,am,a,man] = c

  # tokenize and store info about corpus
  def process(self):
    for i, line in enumerate(self.corpus):
      if i in self.test_indexes:
        self.test_sentences += [line]
        continue
      
      self.train_sentences += [line]
      
      words = ["<s>","<s>","<s>"]
      words += Tokenizer(line)
      words += ["</s>","</s>","</s>"]
      
      if len(words) <= 6:
        continue
      # print(words)
      # break
      
      size = len(words)
      self.total_words += size
      for j, word in enumerate(words):
        # 4-gram cnts
        if j < size - 3:
          if (words[j], words[j+1], words[j+2], words[j+3]) not in self.gramCount:
            self.unique_history_count[(words[j], words[j+1], words[j+2], words[j+3])] += 1
          self.gramCount[(words[j], words[j+1], words[j+2], words[j+3])] += 1

        # Tri-gram
        if j < size - 2:
          if (words[j], words[j+1], words[j+2]) not in self.gramCount:
            self.unique_history_count[(words[j], words[j+1], words[j+2])] += 1
          self.gramCount[(words[j], words[j+1], words[j+2])] += 1

        # Bi-gram
        if j < size - 1:
          if (words[j], words[j+1]) not in self.gramCount:
            self.unique_history_count[(words[j], words[j+1])] += 1
            if (words[j],) not in self.unique_history_count:
              self.unique_history_count[(words[j], )] += 1
          self.gramCount[(words[j], words[j+1])] += 1

        # Uni-gram
        if (words[j],) not in self.gramCount:
            self.unique_history_count[()] += 1
        self.gramCount[(words[j],)] += 1
        
    self.gramCount[()] = self.total_words


In [ ]:
# Returns probability of the sequence 
def KneserNey(corpus, sentence, D=0.75):
  # corpus = corpus class object
  
  ans=0.0
  if(len(sentence)==1):
      if sentence in corpus.gramCount:
          ans=max(corpus.gramCount[sentence]-D, 0)/corpus.gramCount[()]
      ans+=D/(corpus.gramCount[()])
      return ans
  if sentence in corpus.gramCount:
      ans=ans+max(corpus.gramCount[sentence]-D, 0)/corpus.gramCount[sentence[:-1]]
      return ans+D*KneserNey(corpus,sentence[1:])*corpus.unique_history_count[sentence[:-1]]/corpus.gramCount[sentence[:-1]]
  if sentence[:-1] in corpus.gramCount and sentence[1:] in corpus.gramCount:
      return ans+D*KneserNey(corpus,sentence[1:])*corpus.unique_history_count[sentence[:-1]]/corpus.gramCount[sentence[:-1]]
  else:
      return D/(corpus.gramCount[()])

In [ ]:
def Wittenbell(corpus_class, sentence):
    ans=0.0
    
    if(len(sentence)==0):
        return 1/(corpus_class.gramCount[()]+corpus_class.unique_history_count[()])
    if(len(sentence)==1):
        return corpus_class.gramCount[(sentence[-1],)]/(corpus_class.gramCount[()]+corpus_class.unique_history_count[()]) + 1/(corpus_class.gramCount[()]+corpus_class.unique_history_count[()])
    if sentence in corpus_class.gramCount:
        ans=0.0
        ans=ans+((corpus_class.gramCount[sentence]+corpus_class.unique_history_count[sentence[:-1]]*Wittenbell(corpus_class,sentence[1:]))/(corpus_class.gramCount[sentence[:-1]]+corpus_class.unique_history_count[sentence[:-1]]))
        return ans
    if sentence[1:] and sentence[:-1] in corpus_class.gramCount:
        ans=((corpus_class.unique_history_count[sentence[:-1]]*Wittenbell(corpus_class,sentence[1:]))/(corpus_class.gramCount[sentence[:-1]]+corpus_class.unique_history_count[sentence[:-1]]))
        return ans
    if (sentence[-1],) in corpus_class.gramCount:
        return corpus_class.gramCount[(sentence[-1],)]/(corpus_class.gramCount[()]+corpus_class.unique_history_count[()]) + 1/(corpus_class.gramCount[()]+corpus_class.unique_history_count[()])
    return 1/(corpus_class.gramCount[()]+corpus_class.unique_history_count[()])

In [ ]:
file = "./drive/MyDrive/NLP/europarl-corpus.txt"

with open(file) as f:
  europarl = f.readlines()

europarl = Corpus(europarl)
europarl.process()

# europarl test_perplexities
kneserney_perplexities = []
testing_output=""
for j, line in enumerate(europarl.test_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%100 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = KneserNey(europarl, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  testing_output=testing_output+((line.strip())+"\t"+str(final))+"\n"
  kneserney_perplexities.append(final)

testing_output = str((sum(kneserney_perplexities)/len(kneserney_perplexities))) + '\n' + testing_output
file1=open("2019111031-LM1-test-perplexity.txt", "w")
file1.write(testing_output)
file1.close()

print("Testing sentences done")

# europarl train_perplexities
kneserney_perplexities = []
training_output=""
for j, line in enumerate(europarl.train_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%2000 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = KneserNey(europarl, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  training_output=training_output+((line.strip())+"\t"+str(final))+"\n"
  kneserney_perplexities.append(final)

training_output = str((sum(kneserney_perplexities)/len(kneserney_perplexities))) + '\n' + training_output
file1=open("2019111031-LM1-train-perplexity.txt", "w")
file1.write(training_output)
file1.close()

0 Please rise, then, for this minute' s silence.

100 Hence also this joint debate involving transport and environmental management Members.

200 In the field of immigration, borders and asylum, the Commission presented a proposal for a regulation on setting up the Eurodac database, and a proposal for a directive on conditions for admitting third country nationals into Member States of the European Union with a view to reuniting families, as part of a vigorous policy of integrating legal residents in the territory of the European Union.

300 Mr President, Commissioner, I would firstly like to highlight the excellent work carried out by Mrs Berger and the Committee on Legal Affairs and the Internal Market generally, which has incorporated innovative ideas into this directive, which I hope will be approved by the Commission and the Council.

400 It should prevent society from splintering.

500 Unless this CAP is overthrown, no programme can ensure the survival of small and medium-scale f

In [ ]:
file = "./drive/MyDrive/NLP/medical-corpus.txt"

with open(file) as f:
  medical = f.readlines()

medical = Corpus(medical)
medical.process()

# medical test_perplexities
kneserney_perplexities = []
testing_output=""
for j, line in enumerate(medical.test_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%100 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = KneserNey(medical, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  testing_output=testing_output+((line.strip())+"\t"+str(final))+"\n"
  kneserney_perplexities.append(final)

testing_output = str((sum(kneserney_perplexities)/len(kneserney_perplexities))) + '\n' + testing_output
file1=open("2019111031-LM3-test-perplexity.txt", "w")
file1.write(testing_output)
file1.close()

print("Testing sentences done")

# medical train_perplexities
kneserney_perplexities = []
training_output=""
for j, line in enumerate(medical.train_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%2000 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = KneserNey(medical, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  training_output=training_output+((line.strip())+"\t"+str(final))+"\n"
  kneserney_perplexities.append(final)

training_output = str((sum(kneserney_perplexities)/len(kneserney_perplexities))) + '\n' + training_output
file1=open("2019111031-LM3-train-perplexity.txt", "w")
file1.write(training_output)
file1.close()

0 In this study we demonstrate with organoid cultures of fetal rat lungs that the reductive side of 11-hydroxysteroid dehydrogenase (11-HSD) regulates the intracellular activation of GCS.

100 To avoid these disadvantages at high altitude a concept of living at moderately high altitude and training at lower elevations, termed "live high - train low" evolved, opposing the conventional concept of "live high - train high".

200 The median survival time for patients with disseminated malignant melanoma is 5-7 months; it is still not really known whether therapeutic measures prolong survival.

300 fixated metastases with a diameter greater than 6 cm, individually adapted therapy in necessary because of the extremely poor prognosis.

400 The intensified therapy consisted of pravastatin 20-40mg with addition of colestyramine and/or nicotinic acid, if necessary, to achieve an LDL cholesterol <=130mg/dl.

500 Six patients with granuloma faciale, including patients with multiple lesions, were tr

In [ ]:
file = "./drive/MyDrive/NLP/europarl-corpus.txt"

with open(file) as f:
  europarl = f.readlines()

europarl = Corpus(europarl)
europarl.process()

# europarl test_perplexities
wittenbell_perplexities = []
testing_output=""
for j, line in enumerate(europarl.test_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%100 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = Wittenbell(europarl, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  testing_output=testing_output+((line.strip())+"\t"+str(final))+"\n"
  wittenbell_perplexities.append(final)

testing_output = str((sum(wittenbell_perplexities)/len(wittenbell_perplexities))) + '\n' + testing_output
file1=open("2019111031-LM2-test-perplexity.txt", "w")
file1.write(testing_output)
file1.close()

print("Testing sentences done")

# europarl train_perplexities
wittenbell_perplexities = []
training_output=""
for j, line in enumerate(europarl.train_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%2000 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = Wittenbell(europarl, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  training_output=training_output+((line.strip())+"\t"+str(final))+"\n"
  wittenbell_perplexities.append(final)

training_output = str((sum(wittenbell_perplexities)/len(wittenbell_perplexities))) + '\n' + training_output
file1=open("2019111031-LM2-train-perplexity.txt", "w")
file1.write(training_output)
file1.close()

0 That is precisely the time when you may, if you wish, raise this question, i.e. on Thursday prior to the start of the presentation of the report.

100 In conclusion, the document is not particularly satisfactory, and in some aspects is disappointing.

200 The centralised ex ante approval system was doubtlessly originally designed to ensure carefulness but, over the years, it has had the perverse effect of reducing the extent to which managers feel responsible for their decisions.

300 Portugal should not take a leaf out its predecessor' s book.

400 This is why it is useful for the European Union to continue to support the Member States and, if necessary, give additional support under URBAN.

500 The proposed amendment is technical and cannot be interpreted as having as its primary objective the protection of public health.

600 In order to combat distortion of competition and prevent dumping, the Commission has, in my opinion, developed a well-balanced approach which steers a midcou

In [ ]:
file = "./drive/MyDrive/NLP/medical-corpus.txt"

with open(file) as f:
  medical = f.readlines()

medical = Corpus(medical)
medical.process()

# medical test_perplexities
wittenbell_perplexities = []
testing_output=""
for j, line in enumerate(medical.test_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%100 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = Wittenbell(medical, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  testing_output=testing_output+((line.strip())+"\t"+str(final))+"\n"
  wittenbell_perplexities.append(final)

testing_output = str((sum(wittenbell_perplexities)/len(wittenbell_perplexities))) + '\n' + testing_output
file1=open("2019111031-LM4-test-perplexity.txt", "w")
file1.write(testing_output)
file1.close()

print("Testing sentences done")

# medical train_perplexities
wittenbell_perplexities = []
training_output=""
for j, line in enumerate(medical.train_sentences):
  words = ["<s>","<s>","<s>"]
  words += Tokenizer(line)
  words += ["</s>","</s>","</s>"]
  prob = 0

  if j%2000 == 0:
    print(j, line)

  size = len(words)
  for i, word in enumerate(words):
    if i < size-3:
      kn = Wittenbell(medical, (words[i], words[i+1], words[i+2], words[i+3]))
      log_kn = math.log(kn)
      prob += log_kn
  final = math.e**(-prob/size)
  training_output=training_output+((line.strip())+"\t"+str(final))+"\n"
  wittenbell_perplexities.append(final)

training_output = str((sum(wittenbell_perplexities)/len(wittenbell_perplexities))) + '\n' + training_output
file1=open("2019111031-LM4-train-perplexity.txt", "w")
file1.write(training_output)
file1.close()

0 On the other hand, GCS induce the reductive activity of 11-HSD and, therefore, they increase their own activation by positive feedback.

100 Abdominal aortic cross-clamping changes circulation, nutritional supply of the lower extremities and thus V.O2 and V.CO2.

200 Cleft lip and palates (CLP) are caused by a variety of factors.

300 In view of the poor visual and ocular prognosis in severe retinal vessel occlusion, controlled clinical trials are needed to determine the benefit of thrombolysis in the management of this disease.

400 The average time from loss of consciousness to tracheal intubation was 164 +/- 8 s. Conclusions.

500 Typ II: The leading thoracic injury is a lung contusion which occurs in up to 50 % of the cases.

600 Magnetic resonance tomography (MRT) has become the most important method in the workup of infantile cerebral complications after primary sonography.

700 Delayed diagnoses were defined as the difference between the ISS at discharge and the ISS at complet

In [ ]:
inp = input("Usage: <smoothing type> <path to corpus>\n")
inp = inp.split()

if len(inp) != 2 or (inp[0]!='k' and inp[0]!='w') or os.path.exists(inp[1]) == False:
  print("Error")
else:
  with open(file) as f:
    corpus = f.readlines()
  corpus = Corpus(corpus)
  corpus.process()

  st = input("Enter sentence: ")
  final_prob = 1
  for j, line in enumerate(st):
    words = ["<s>","<s>","<s>"]
    words += Tokenizer(line)
    words += ["</s>","</s>","</s>"]

    size = len(words)
    for i, word in enumerate(words):
      if i < size-3:
        if inp[0] == 'w':
          p = Wittenbell(corpus, (words[i], words[i+1], words[i+2], words[i+3]))
        else:
          p = KneserNey(corpus, (words[i], words[i+1], words[i+2], words[i+3]))
        final_prob *= p
  print(f'\nProbability: {final_prob}')

Usage: <smoothing type> <path to corpus>
w ./drive/MyDrive/NLP/medical-corpus.txt
Enter sentence: I am a man

Probability: 6.964944857213407e-166
